<a href="https://colab.research.google.com/github/farrelrassya/teachingMLDL/blob/main/01.%20Machine%20Learning/02.%20Week%202/Notebook/02_Linear_Regression_with_Lasso_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
dataset = pd.read_csv('Melbourne_housing_FULL.csv')
dataset.head(5)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [3]:
dataset.nunique()

,0
Suburb,351
Address,34009
Rooms,12
Type,3
Price,2871
Method,9
SellerG,388
Date,78
Distance,215
Postcode,211


In [4]:
cols_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount',
               'Distance', 'CouncilArea', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Price']
dataset = dataset[cols_to_use]
dataset.head()

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.0,NaN,NaN
1,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,202.0,NaN,1480000.0
2,Abbotsford,2,h,S,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,0.0,156.0,79.0,1035000.0
3,Abbotsford,3,u,VB,Rounds,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,1.0,0.0,NaN,NaN
4,Abbotsford,3,h,SP,Biggin,Northern Metropolitan,4019.0,2.5,Yarra City Council,3.0,2.0,0.0,134.0,150.0,1465000.0


In [5]:
dataset.shape

(34857, 15)

In [6]:
dataset.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Method,0
SellerG,0
Regionname,3
Propertycount,3
Distance,1
CouncilArea,3
Bedroom2,8217


In [7]:
# Some feature's missing values can be treated as zero (another class for NA values or absence of that feature)
# like 0 for Propertycount, Bedroom2 will refer to other class of NA values
# like 0 for Car feature will mean that there's no car parking feature with house
cols_to_fill_zero = ['Propertycount', 'Distance', 'Bedroom2', 'Bathroom', 'Car']
dataset[cols_to_fill_zero] = dataset[cols_to_fill_zero].fillna(0)

# other continuous features can be imputed with mean for faster results since our focus is on Reducing overfitting
# using Lasso and Ridge Regression
dataset['Landsize'] = dataset['Landsize'].fillna(dataset.Landsize.mean())
dataset['BuildingArea'] = dataset['BuildingArea'].fillna(dataset.BuildingArea.mean())

In [8]:
dataset.dropna(inplace=True)

In [9]:
dataset.shape

(27244, 15)

In [10]:
dataset = pd.get_dummies(dataset, drop_first=True)

In [11]:
dataset.head()

,Rooms,Propertycount,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Price,Suburb_Aberfeldie,...,CouncilArea_Moorabool Shire Council,CouncilArea_Moreland City Council,CouncilArea_Nillumbik Shire Council,CouncilArea_Port Phillip City Council,CouncilArea_Stonnington City Council,CouncilArea_Whitehorse City Council,CouncilArea_Whittlesea City Council,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council
1,2,4019.0,2.5,2.0,1.0,1.0,202.0,160.2564,1480000.0,False,...,False,False,False,False,False,False,False,False,True,False
2,2,4019.0,2.5,2.0,1.0,0.0,156.0,79.0000,1035000.0,False,...,False,False,False,False,False,False,False,False,True,False
4,3,4019.0,2.5,3.0,2.0,0.0,134.0,150.0000,1465000.0,False,...,False,False,False,False,False,False,False,False,True,False
5,3,4019.0,2.5,3.0,2.0,1.0,94.0,160.2564,850000.0,False,...,False,False,False,False,False,False,False,False,True,False
6,4,4019.0,2.5,3.0,1.0,2.0,120.0,142.0000,1600000.0,False,...,False,False,False,False,False,False,False,False,True,False


In [12]:
# X is the DataFrame without the 'Price' column
X = dataset.drop('Price', axis=1)

# y is the target variable, which contains only the 'Price' column
y = dataset['Price']


In [13]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state=2)

In [14]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(train_X, train_y)

Coefficient of Determination (R-squared) for Regression
$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$

- $R^2$: Coefficient of Determination, a metric that indicates the proportion of variance in the dependent variable that is predictable from the independent variable(s).
- $y_i$: The actual value for the $i$-th observation.
- $\hat{y}_i$: The predicted value for the $i$-th observation, generated by the regression model.
- $\bar{y}$: The mean (average) of the actual values $y_i$.
- $\sum_{i=1}^{n} (y_i - \hat{y}_i)^2$: The sum of squared residuals (Residual Sum of Squares, RSS), representing the unexplained variance by the model.
- $\sum_{i=1}^{n} (y_i - \bar{y})^2$: The total sum of squares (Total Sum of Squares, TSS), representing the total variance in the actual data.


In [15]:
model.score(train_X, train_y)

0.6827792395792723

In [16]:
model.score(test_X, test_y)

0.1385368316165222

Understanding the Range of the Coefficient of Determination ($R^2$)

- **Range of $R^2$**: The value of $R^2$ can range from **$-\infty$ to $1$**.

  - **$R^2 = 1$**: Indicates a perfect fit. The regression predictions perfectly match the actual data points.
  
  - **$R^2 = 0$**: Suggests that the model does not explain any of the variability in the response variable. The predictions are no better than using the mean of the actual values.
  
  - **$0 < R^2 < 1$**: The model explains a portion of the variance in the response variable. A higher value indicates a better fit.
  
  - **$R^2 < 0$**: Implies that the model performs worse than a horizontal line (mean of the data). This can occur when the model is a poor fit or when the mean provides a better prediction than the model.

- **Interpretation**:

  - **$R^2$ as Explained Variance**: The value of $R^2$ represents the proportion of variance in the dependent variable that is predictable from the independent variable(s).

  - **High $R^2$ Values**: An $R^2$ close to $1$ means that a large proportion of the variance in the response variable is accounted for by the model.

  - **Low $R^2$ Values**: An $R^2$ near $0$ indicates that the model does not explain much of the variance in the response variable.

- **Mathematical Boundaries**:

  - **Upper Bound**: $R^2 \leq 1$ because the model cannot explain more variance than there is in the data.

  - **Lower Bound**: While the theoretical lower limit is $-\infty$, in practice, extremely negative values suggest a model that is a very poor fit.

- **Important Notes**:

  - **Negative $R^2$ Values**: These occur when the chosen model does not follow the trend of the data, often due to incorrect model specifications or overfitting.

  - **Adjusted $R^2$**: When comparing models with different numbers of predictors, it's advisable to use the adjusted $R^2$ to account for the number of variables in the model.

- **Equation for Adjusted $R^2$**:

  $$
  \text{Adjusted } R^2 = 1 - \left(1 - R^2\right) \times \frac{n - 1}{n - p - 1}
  $$

  - $\text{Adjusted } R^2$: Adjusted Coefficient of Determination.
  - $n$: Total number of observations.
  - $p$: Number of independent variables (predictors) in the model.

- **Example Interpretation**:

  - **$R^2 = 0.85$**: The model explains 85% of the variability in the response variable.

  - **$R^2 = -0.2$**: The model does not follow the trend of the data and performs worse than simply using the mean of the response variable.

- **Visual Representation**:

  - A scatter plot of actual vs. predicted values can help visualize how well the model fits the data corresponding to different $R^2$ values.

- **Limitations**:

  - **Not Indicative of Causation**: A high $R^2$ does not imply that the independent variables cause the changes in the dependent variable.

  - **Affected by Outliers**: Outliers can significantly impact the $R^2$ value, leading to misleading interpretations.

- **Considerations**:

  - Always analyze $R^2$ in conjunction with residual plots and other diagnostic measures to assess model fit comprehensively.

  - For non-linear models or models without an intercept, $R^2$ might not be the best measure of fit.



In [17]:
from sklearn import linear_model
lasso_reg = linear_model.Lasso(alpha=50, max_iter=100, tol=0.1)
lasso_reg.fit(train_X, train_y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.327e+15, tolerance: 7.963e+14
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=50, max_iter=100, tol=0.1)

Ridge Regression Objective Function
$$
J(\beta) = \sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2 + \lambda \sum_{j=1}^{p} \beta_j^2
$$

- $J(\beta)$: The cost function to be minimized in Ridge regression.
- $n$: The total number of observations or data points in the dataset.
- $p$: The number of features or predictor variables.
- $y_i$: The actual value for the $i$-th observation.
- $\beta_0$: The intercept term (bias) of the model.
- $\beta_j$: The coefficient for the $j$-th feature.
- $x_{ij}$: The value of the $j$-th feature for the $i$-th observation.
- $\sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2$: The sum of squared residuals between the actual and predicted values.
- $\lambda$: The regularization parameter controlling the strength of the penalty term.
- $\sum_{j=1}^{p} \beta_j^2$: The L2 norm (sum of squares) of the coefficients, which shrinks the coefficients but does not set them exactly to zero.


Lasso Regression Objective Function
$$
J(\beta) = \sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2 + \lambda \sum_{j=1}^{p} |\beta_j|
$$

- $J(\beta)$: The cost function to be minimized in Lasso regression.
- $n$: The total number of observations or data points in the dataset.
- $p$: The number of features or predictor variables.
- $y_i$: The actual value for the $i$-th observation.
- $\beta_0$: The intercept term (bias) of the model.
- $\beta_j$: The coefficient for the $j$-th feature.
- $x_{ij}$: The value of the $j$-th feature for the $i$-th observation.
- $\sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2$: The sum of squared residuals between the actual and predicted values.
- $\lambda$: The regularization parameter controlling the strength of the penalty term.
- $\sum_{j=1}^{p} |\beta_j|$: The L1 norm (sum of absolute values) of the coefficients, promoting sparsity in the model by driving some coefficients to zero.

In [18]:
lasso_reg.score(test_X, test_y)

0.6636111369404489

In [19]:
lasso_reg.score(train_X, train_y)

0.6766985624766824

In [20]:
from sklearn.linear_model import Ridge
ridge_reg= Ridge(alpha=50, max_iter=100, tol=0.1)
ridge_reg.fit(train_X, train_y)

Ridge(alpha=50, max_iter=100, tol=0.1)

In [21]:
ridge_reg.score(train_X, train_y)

0.6622376739684328

In [22]:
ridge_reg.score(test_X, test_y)

0.6670848945194958

% Understanding Ridge and Lasso Regression

---

### **Ridge Regression**

- **Objective Function**:

  $$
  J_{\text{Ridge}}(\beta) = \sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2 \quad + \quad \lambda \sum_{j=1}^{p} \beta_j^2
  $$

  - $J_{\text{Ridge}}(\beta)$: The cost function to minimize in Ridge Regression.
  - $n$: Total number of observations.
  - $p$: Number of predictor variables.
  - $y_i$: Actual value of the response variable for observation $i$.
  - $\beta_0$: Intercept term (bias).
  - $\beta_j$: Coefficient for the $j$-th predictor variable.
  - $x_{ij}$: Value of the $j$-th predictor for observation $i$.
  - $\lambda$: Regularization parameter controlling the strength of the penalty term.
  - $\sum_{j=1}^{p} \beta_j^2$: The L2 norm (sum of squares) of the coefficients.

- **Interpretation**:

  - **Penalty Term**: The $\lambda \sum_{j=1}^{p} \beta_j^2$ term penalizes large coefficients, shrinking them towards zero.
  - **Effect**: Reduces model complexity and multicollinearity without eliminating variables.
  - **Use Case**: Useful when predictors are numerous and highly correlated.

- **Mathematical Boundaries**:

  - **$\lambda = 0$**: Ridge Regression reduces to Ordinary Least Squares (no regularization).
  - **$\lambda \rightarrow \infty$**: Coefficients $\beta_j$ approach zero but never exactly zero.

- **Important Notes**:

  - **Does Not Perform Feature Selection**: Coefficients are shrunk but not set to zero.
  - **Bias-Variance Trade-off**: Introduces bias to reduce variance, potentially improving generalization.

- **Limitations**:

  - **Interpretability**: All features remain in the model, which may not simplify interpretation.
  - **Requires Feature Scaling**: Standardizing predictors is necessary to ensure fair penalization.

- **Considerations**:

  - **Selecting $\lambda$**: Typically determined through cross-validation.
  - **Correlation Among Predictors**: Ridge Regression is effective in handling multicollinearity.

---

### **Lasso Regression**

- **Objective Function**:

  $$
  J_{\text{Lasso}}(\beta) = \sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2 \quad + \quad \lambda \sum_{j=1}^{p} |\beta_j|
  $$

  - $J_{\text{Lasso}}(\beta)$: The cost function to minimize in Lasso Regression.
  - $n$: Total number of observations.
  - $p$: Number of predictor variables.
  - $y_i$: Actual value of the response variable for observation $i$.
  - $\beta_0$: Intercept term (bias).
  - $\beta_j$: Coefficient for the $j$-th predictor variable.
  - $x_{ij}$: Value of the $j$-th predictor for observation $i$.
  - $\lambda$: Regularization parameter controlling the strength of the penalty term.
  - $\sum_{j=1}^{p} |\beta_j|$: The L1 norm (sum of absolute values) of the coefficients.

- **Interpretation**:

  - **Penalty Term**: The $\lambda \sum_{j=1}^{p} |\beta_j|$ term encourages sparsity by penalizing the absolute size of coefficients.
  - **Effect**: Some coefficients may become exactly zero, effectively performing feature selection.
  - **Use Case**: Ideal when you suspect only a few predictors are significant.

- **Mathematical Boundaries**:

  - **$\lambda = 0$**: Lasso Regression reduces to Ordinary Least Squares (no regularization).
  - **$\lambda \rightarrow \infty$**: Coefficients $\beta_j$ are driven towards zero; more coefficients become zero.

- **Important Notes**:

  - **Feature Selection**: Lasso can simplify models by eliminating less important features.
  - **Bias Introduction**: Adds bias to reduce variance, similar to Ridge Regression.

- **Limitations**:

  - **Variable Selection Bias**: May arbitrarily select one variable from a group of correlated predictors.
  - **Model Stability**: Can be sensitive to small changes in the data.

- **Considerations**:

  - **Selecting $\lambda$**: Use cross-validation to find the optimal value.
  - **Data Preprocessing**: Standardization of predictors is crucial.

---

### **Comparison Between Ridge and Lasso Regression**

- **Penalty Type**:

  - **Ridge**: Uses L2 norm ($\beta_j^2$) penalization.
  - **Lasso**: Uses L1 norm ($|\beta_j|$) penalization.

- **Effect on Coefficients**:

  - **Ridge**: Shrinks coefficients but does not set them to zero.
  - **Lasso**: Can shrink coefficients to exactly zero, performing variable selection.

- **Use Cases**:

  - **Ridge**: When all predictors are expected to contribute to the outcome.
  - **Lasso**: When only a subset of predictors are believed to be important.

---

### **Mathematical Boundaries and Properties**

- **Regularization Parameter ($\lambda$)**:

  - Controls the trade-off between fitting the data and keeping the model coefficients small.
  - **Higher $\lambda$**: More regularization, greater penalty on coefficients.

- **Bias-Variance Trade-off**:

  - **Ridge and Lasso** introduce bias to reduce variance.
  - **Goal**: Improve model's predictive performance on new data.

---

### **Important Notes**

- **Feature Standardization**:

  - Essential for both Ridge and Lasso to ensure fair penalization across all coefficients.
  - Typically involves scaling features to have zero mean and unit variance.

- **Model Selection**:

  - **Cross-Validation**: Commonly used to select the optimal $\lambda$.
  - **Grid Search**: Testing multiple $\lambda$ values to minimize validation error.

---

### **Limitations**

- **Ridge Regression**:

  - Does not perform feature selection; all predictors remain in the model.
  - May not simplify model interpretation when dealing with many predictors.

- **Lasso Regression**:

  - Can be unstable when predictors are highly correlated.
  - May arbitrarily select one predictor over others in a correlated group.

---

### **Considerations**

- **When to Use Ridge**:

  - When you have many predictors, and multicollinearity is a concern.
  - When you prefer to keep all predictors in the model.

- **When to Use Lasso**:

  - When you expect that only a few predictors are significant.
  - When model interpretability and simplicity are important.

- **Elastic Net Regression**:

  - Combines L1 and L2 penalties.
  - **Objective Function**:

    $$
    J_{\text{ElasticNet}}(\beta) = \sum_{i=1}^{n} \left( y_i - \beta_0 - \sum_{j=1}^{p} \beta_j x_{ij} \right)^2 + \lambda_1 \sum_{j=1}^{p} |\beta_j| + \lambda_2 \sum_{j=1}^{p} \beta_j^2
    $$

  - **Use Case**: When you want feature selection (like Lasso) and need to handle multicollinearity (like Ridge).

---

### **Practical Tips**

- **Feature Engineering**:

  - Remove or combine correlated predictors before applying Lasso.
  - Use domain knowledge to select relevant features.

- **Interpreting Coefficients**:

  - **Ridge**: Coefficients are shrunk but all remain; interpretation involves considering all predictors.
  - **Lasso**: Zero coefficients indicate non-important features; simplifies interpretation.

- **Model Evaluation**:

  - Use metrics like Mean Squared Error (MSE) or Mean Absolute Error (MAE) on validation data.
  - Check for overfitting by comparing training and validation errors.

---

### **Conclusion**

- **Choice Between Ridge and Lasso**:

  - **Ridge Regression**: Preferred when you believe all features contribute somewhat to the outcome and multicollinearity is an issue.
  - **Lasso Regression**: Preferred when you aim for a simpler model with feature selection.

- **Key Takeaway**:

  - Both methods aim to prevent overfitting by adding a penalty term to the loss function.
  - The choice depends on the specific dataset and the goals of the analysis.

---

### **Visual Representation**

- **Ridge Regression**:

  - Penalty contours are circular (in two dimensions), leading to coefficients being shrunk proportionally.

- **Lasso Regression**:

  - Penalty contours are diamond-shaped, encouraging coefficients to reach zero at the corners.

---

### **Additional Notes**

- **Model Stability**:

  - **Lasso** can be less stable due to its tendency to select variables.
  - **Ridge** tends to be more stable in terms of coefficient estimates.

- **Computational Efficiency**:

  - Both methods are computationally efficient and can handle large datasets.

---

By understanding the differences, strengths, and limitations of Ridge and Lasso Regression, you can make informed decisions about which technique is more suitable for your specific regression problems.
